In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [2]:
import os
from pathlib import Path

folder = "colab/pytorch"
project_dir = "Auto_Keras"

base_path = Path("/content/gdrive/My Drive/")
project_path = base_path / folder / project_dir
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"현재 디렉토리 위치: {os.getcwd()}")

현재 디렉토리 위치: /content/gdrive/My Drive/colab/pytorch/Auto_Keras


In [16]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import re 
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd

In [23]:
data = pd.read_csv('/content/gdrive/My Drive/colab/pytorch/Auto_Keras/Data/car_scrape.csv')

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451 entries, 0 to 1450
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1451 non-null   object
 1   odometer      1451 non-null   int64 
 2   location      1451 non-null   object
 3   isimported    1451 non-null   object
 4   engine        1451 non-null   object
 5   transmission  1451 non-null   object
 6   fuel          1451 non-null   object
 7   paint         1451 non-null   object
 8   price         1451 non-null   int64 
 9   year          1451 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 113.5+ KB


In [25]:
data = data[data['year'] > 1900]

In [33]:
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i+1
            label_maps[col] = label_map
    
    return label_maps

def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown'])
    
    return dataframe

In [34]:
le = make_label_map(data)
data = label_encoder(data, le)

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data[[col for col in data.columns if col not in ['price']]],
                                                    data['price'], test_size = 0.33)

In [37]:
from sklearn.svm import SVR

model = SVR()
model.fit(x_train, y_train)

SVR()

In [38]:
def nmae(true, pred):

    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    
    return score

In [39]:
y_pred = model.predict(x_test)

In [40]:
svr_nmae = nmae(y_test, y_pred)

In [41]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 6.0 MB/s 
     |████████████████████████████████| 135 kB 47.7 MB/s 
     |████████████████████████████████| 1.6 MB 65.2 MB/s 


In [42]:
import autokeras as ak

In [43]:
reg = ak.StructuredDataRegressor(overwrite=True, max_trials=5)

In [44]:
reg.fit(x_train, y_train, 100)

Trial 5 Complete [00h 00m 28s]
val_loss: 145487000764416.0

Best val_loss So Far: 124868146233344.0
Total elapsed time: 00h 02m 24s
Epoch 1/100
31/31 [==============================] - 1s 4ms/step - loss: 187482184351744.0000 - mean_squared_error: 187482184351744.0000
Epoch 2/100
31/31 [==============================] - 0s 4ms/step - loss: 187482134020096.0000 - mean_squared_error: 187482134020096.0000
Epoch 3/100
31/31 [==============================] - 0s 3ms/step - loss: 187482016579584.0000 - mean_squared_error: 187482016579584.0000
Epoch 4/100
31/31 [==============================] - 0s 4ms/step - loss: 187481429377024.0000 - mean_squared_error: 187481429377024.0000
Epoch 5/100
31/31 [==============================] - 0s 4ms/step - loss: 187479818764288.0000 - mean_squared_error: 187479818764288.0000
Epoch 6/100
31/31 [==============================] - 0s 3ms/step - loss: 187476144553984.0000 - mean_squared_error: 187476144553984.0000
Epoch 7/100
31/31 [===========================

In [45]:
autokeras_model = reg.export_model()

In [46]:
autokeras_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 9)                0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 9)                19        
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                320       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [47]:
tf.keras.backend.clear_session()

model = tf.keras.models.load_model('./structured_data_regressor/best_model/')

In [48]:
model.fit(x_train, y_train, epochs=100, validation_split=0.3)

Epoch 1/100
22/22 [==============================] - 2s 43ms/step - loss: 81311372935168.0000 - mean_squared_error: 81311372935168.0000 - val_loss: 98444450463744.0000 - val_mean_squared_error: 98444450463744.0000
Epoch 2/100
22/22 [==============================] - 0s 12ms/step - loss: 81165008502784.0000 - mean_squared_error: 81165008502784.0000 - val_loss: 98312078229504.0000 - val_mean_squared_error: 98312078229504.0000
Epoch 3/100
22/22 [==============================] - 0s 14ms/step - loss: 81085920706560.0000 - mean_squared_error: 81085920706560.0000 - val_loss: 98261931130880.0000 - val_mean_squared_error: 98261931130880.0000
Epoch 4/100
22/22 [==============================] - 0s 13ms/step - loss: 81011555696640.0000 - mean_squared_error: 81011555696640.0000 - val_loss: 98222177517568.0000 - val_mean_squared_error: 98222177517568.0000
Epoch 5/100
22/22 [==============================] - 0s 14ms/step - loss: 80916697317376.0000 - mean_squared_error: 80916697317376.0000 - val_lo

In [60]:
autokeras_y_pred = model.predict(x_test)

In [61]:
autokeras_y_pred = autokeras_y_pred.squeeze()

In [62]:
autokeras_nmae = nmae(y_test, autokeras_y_pred)

In [63]:
print(f'SVR 모델 NMAE: {svr_nmae}')
print(f'AutoKeras NMAE: {autokeras_nmae}')

SVR 모델 NMAE: 0.7284698727223285
AutoKeras NMAE: 0.7103537516393555
